# Pump it Up: Data Mining the Water Table

### Can you predict which water pumps are faulty?

Using data from Taarifa and the Tanzanian Ministry of Water, we are trying to predict which pumps are functional, which need some repairs, and which don't work at all based on a number of variables about what kind of pump is operating, when it was installed, and how it is managed. 

A smart understanding of which waterpoints will fail can improve maintenance operations and ensure that clean, potable water is available to communities across Tanzania.

In [38]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler as ss
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV


pd.set_option('display.max_columns', None)

# machine learning
#Trees    
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier

#Ensemble Methods
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.experimental import enable_hist_gradient_boosting  # explicitly require this experimental feature
from sklearn.ensemble import HistGradientBoostingClassifier # now you can import normally from ensemble
from lightgbm import LGBMClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier

#Gaussian Processes
from sklearn.gaussian_process import GaussianProcessClassifier
    
#GLM
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn.linear_model import Perceptron   
    
#Nearest Neighbor
from sklearn.neighbors import KNeighborsClassifier
    
#SVM
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.svm import NuSVC
    
# xgboost: http://xgboost.readthedocs.io/en/latest/model.html
# from xgboost import XGBClassifier

#Discriminant Analysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

 #Navies Bayes
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB

# metrics
from sklearn.metrics import accuracy_score, confusion_matrix

# PCA
from sklearn import decomposition

print("Setup Complete")

Setup Complete


In [39]:
# Specify the path of the CSV file to read
train_df_final = pd.read_csv("train_df_final.csv")
X_test_final = pd.read_csv("X_test_final.csv")

In [40]:
X_test_final.shape

(14850, 30)

In [41]:
train_df_final.shape

(59400, 31)

# Model building
## Train/test splitting

In [42]:
X = train_df_final.drop("label",axis=1)
y = train_df_final["label"]

In [57]:
# Create training and test sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2, stratify=y, random_state=42)

In [58]:
X.isnull().values.any()

False

This stratify parameter makes a split so that the proportion of values in the sample produced will be the same as the proportion of values provided to parameter stratify.

For example, if variable y is a binary categorical variable with values 0 and 1 and there are 25% of zeros and 75% of ones, stratify=y will make sure that your random split has 25% of 0's and 75% of 1's.

## Standard Scaling

The idea behind StandardScaler is that it will transform your data such that its distribution will have a mean value 0 and standard deviation of 1.

In case of multivariate data, this is done feature-wise (in other words independently for each column of the data).

Given the distribution of the data, each value in the dataset will have the mean value subtracted, and then divided by the standard deviation of the whole dataset (or feature in the multivariate case).

https://stackoverflow.com/questions/40758562/can-anyone-explain-me-standardscaler

In [59]:
sc = ss()
X_train = sc.fit_transform(X_train)
X_valid = sc.transform(X_valid)
X_test = sc.transform(X_test_final)

## Principle component analysis (PCA)

In [61]:
# Make an instance of the Model
pca = decomposition.PCA(.95)

In [62]:
pca.fit(X_train)

PCA(copy=True, iterated_power='auto', n_components=0.95, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [63]:
pca.n_components_

58

In [65]:
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)
X_valid_pca = pca.transform(X_valid)

## Model selection
### Try different models

**TO DO**: combine all models in a loop

Try K-nearest neighbors? If possible to use long and lat features to define areas like on the map above with hue = labels -> it's logical that pumps in the same area function similary...? 

In [ ]:
# #Common Model Algorithms
# from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
# from xgboost import XGBClassifier
# from sklearn import model_selection


# #Machine Learning Algorithm (MLA) Selection and Initialization
# MLA = [
#     #Ensemble Methods
#   #  ensemble.AdaBoostClassifier(),
#     ensemble.BaggingClassifier(),
#     ensemble.ExtraTreesClassifier(),
#     ensemble.GradientBoostingClassifier(),
#     ensemble.RandomForestClassifier(),

#     #Gaussian Processes
#     gaussian_process.GaussianProcessClassifier(),
    
#     #GLM
#     linear_model.LogisticRegressionCV(),
#     linear_model.PassiveAggressiveClassifier(),
#     linear_model.RidgeClassifierCV(),
#     linear_model.SGDClassifier(),
#     linear_model.Perceptron(),
    
#     #Navies Bayes
#     naive_bayes.BernoulliNB(),
#     naive_bayes.GaussianNB(),
    
#     #Nearest Neighbor
#     neighbors.KNeighborsClassifier(),
    
#     #SVM
#     svm.SVC(probability=True),
#     svm.NuSVC(probability=True),
#     svm.LinearSVC(),
    
#     #Trees    
#     tree.DecisionTreeClassifier(),
#     tree.ExtraTreeClassifier(),
    
#     #Discriminant Analysis
#     discriminant_analysis.LinearDiscriminantAnalysis(),
#     discriminant_analysis.QuadraticDiscriminantAnalysis(),

    
#     #xgboost: http://xgboost.readthedocs.io/en/latest/model.html
#     XGBClassifier()    
#     ]

# #split dataset in cross-validation with this splitter class: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ShuffleSplit.html#sklearn.model_selection.ShuffleSplit
# #note: this is an alternative to train_test_split
# cv_split = model_selection.ShuffleSplit(n_splits = 10, test_size = .3, train_size = .7, random_state = 0 ) # run model 10x with 70/30 split 

# #create table to compare MLA metrics
# MLA_columns = ['MLA Name', 'MLA Parameters','MLA Train Accuracy Mean', 'MLA Test Accuracy Mean', 'MLA Test Accuracy 3*STD' ,'MLA Time']
# MLA_compare = pd.DataFrame(columns = MLA_columns)

# # #create table to compare MLA predictions
# # MLA_predict = y

# #index through MLA and save performance to table
# row_index = 0
# for alg in MLA:

#     #set name and parameters
#     MLA_name = alg.__class__.__name__
#     MLA_compare.loc[row_index, 'MLA Name'] = MLA_name
#     MLA_compare.loc[row_index, 'MLA Parameters'] = str(alg.get_params())
    
#     #score model with cross validation: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate
#     cv_results = model_selection.cross_validate(alg, X, y, cv  = cv_split,return_train_score=True)

#     MLA_compare.loc[row_index, 'MLA Time'] = cv_results['fit_time'].mean()
#     MLA_compare.loc[row_index, 'MLA Train Accuracy Mean'] = cv_results['train_score'].mean()
#     MLA_compare.loc[row_index, 'MLA Test Accuracy Mean'] = cv_results['test_score'].mean()   
#     #if this is a non-bias random sample, then +/-3 standard deviations (std) from the mean, should statistically capture 99.7% of the subsets
#     MLA_compare.loc[row_index, 'MLA Test Accuracy 3*STD'] = cv_results['test_score'].std()*3   #let's know the worst that can happen!
    

# #     #save MLA predictions - see section 6 for usage
# #     alg.fit(X, y)
# #     MLA_predict[MLA_name] = alg.predict(X)
    
#     row_index+=1

    
# #print and sort table: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_values.html
# MLA_compare.sort_values(by = ['MLA Test Accuracy Mean'], ascending = False, inplace = True)
# MLA_compare
# #MLA_predict

#### Trees

In [ ]:
# Decision Tree

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)
y_pred = decision_tree.predict(X_valid)

acc_decision_tree = round(accuracy_score(y_valid,y_pred) * 100, 2)
acc_decision_tree

In [ ]:
# Extra Tree

extra_tree = DecisionTreeClassifier()
extra_tree.fit(X_train, y_train)
y_pred = extra_tree.predict(X_valid)

acc_extra_tree = round(accuracy_score(y_valid,y_pred) * 100, 2)
acc_extra_tree

#### Ensembles

In [67]:
# Random Forest

rfc = RandomForestClassifier(criterion='entropy', n_estimators = 1000,min_samples_split=8,random_state=42,verbose=5)
rfc.fit(X_train_pca, y_train)

y_pred = rfc.predict(X_valid_pca)

acc_rfc = round(accuracy_score(y_valid,y_pred) * 100, 2)
acc_rfc

77.19

In [68]:
# GradientBoostingClassifier

GB = GradientBoostingClassifier(n_estimators=100, learning_rate=0.075, 
                                max_depth=14,max_features=1.0,
                                min_samples_leaf=14, verbose=10)

GB.fit(X_train_pca, y_train)     
y_pred = GB.predict(X_valid_pca)

acc_GB = round(accuracy_score(y_valid,y_pred) * 100, 2)
acc_GB


      Iter       Train Loss   Remaining Time 
         1       39072.4027           36.35m
         2       36456.2492           37.71m
         3       34245.8275           37.64m
         4       32326.4488           37.41m
         5       30635.3955           37.00m
         6       29119.2526           36.79m
         7       27733.2958           36.50m
         8       26499.9202           36.11m
         9       25380.9488           35.65m
        10       24348.9807           35.33m
        11       23418.3368           34.93m
        12       22565.3432           34.49m
        13       21742.2631           34.20m
        14       20984.1346           34.09m
        15       20275.8523           33.72m
        16       19636.4608           33.29m
        17       19032.3979           32.85m
        18       18472.9692           32.36m
        19       17961.3823           31.83m
        20       17457.6397           31.39m
        21       16970.5176           30.96m
        2

77.04

In [70]:
# Histogram-based Gradient Boosting Classification Tree.

#This estimator is much faster than GradientBoostingClassifier for big datasets (n_samples >= 10 000).


HGB = HistGradientBoostingClassifier(learning_rate=0.075, loss='categorical_crossentropy', 
                                               max_depth=8, min_samples_leaf=15)

HGB = HGB.fit(X_train_pca, y_train)

y_pred = HGB.predict(X_valid_pca)

acc_HGB = round(accuracy_score(y_valid,y_pred) * 100, 2)
acc_HGB

76.28

In [71]:
# LightGBM 

#is another fast tree based gradient boosting algorithm, which supports GPU, and parallel learning.


LGB = LGBMClassifier(objective='multiclass', learning_rate=0.75, num_iterations=100, 
                     num_leaves=40, random_state=123, max_depth=15)

LGB.fit(X_train_pca, y_train)
y_pred = LGB.predict(X_valid_pca)

acc_LGB = round(accuracy_score(y_valid,y_pred) * 100, 2)
acc_LGB

/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


75.98

In [ ]:
# AdaBoost classifier

AB = AdaBoostClassifier(n_estimators=100, learning_rate=0.075)
AB.fit(X_train, y_train)     
y_pred = AB.predict(X_valid)

acc_AB = round(accuracy_score(y_valid,y_pred) * 100, 2)
acc_AB

In [ ]:
# BaggingClassifier

BC = BaggingClassifier(n_estimators=100)
BC.fit(X_train, y_train)     
y_pred = BC.predict(X_valid)

acc_BC = round(accuracy_score(y_valid,y_pred) * 100, 2)
acc_BC

In [ ]:
# ExtraTreesClassifier

ETC = ExtraTreesClassifier(n_estimators=100)
ETC.fit(X_train, y_train)     
y_pred = ETC.predict(X_valid)

acc_ETC = round(accuracy_score(y_valid,y_pred) * 100, 2)
acc_ETC

#### GLM

In [ ]:
# Logistic Regression for multilabel classification

# https://acadgild.com/blog/logistic-regression-multiclass-classification
# https://medium.com/@jjw92abhi/is-logistic-regression-a-good-multi-class-classifier-ad20fecf1309

LG = LogisticRegression(solver="lbfgs", multi_class="multinomial")
LG.fit(X_train, y_train)     
y_pred = LG.predict(X_valid)

acc_LG = round(accuracy_score(y_valid,y_pred) * 100, 2)
acc_LG

We can use Logistic Regression to validate our assumptions and decisions for feature creating and completing goals. This can be done by calculating the coefficient of the features in the decision function.

Positive coefficients increase the odds of the response (and thus increase the probability), and negative coefficients decrease the odds of the response (and thus decrease the probability).

In [ ]:
coeff_df = pd.DataFrame(train_df_final.columns.delete(0))
coeff_df.columns = ['Feature']
coeff_df["Correlation"] = pd.Series(LG.coef_[0])

coeff_df.sort_values(by='Correlation', ascending=False)

In [ ]:
# PassiveAggressiveClassifier

PAC = PassiveAggressiveClassifier()
PAC.fit(X_train, y_train)
y_pred = PAC.predict(X_valid)

acc_PAC = round(accuracy_score(y_valid,y_pred) * 100, 2)
acc_PAC

In [ ]:
# RidgeClassifierCV

RC = RidgeClassifierCV()
RC.fit(X_train, y_train)
y_pred = RC.predict(X_valid)

acc_RC = round(accuracy_score(y_valid,y_pred) * 100, 2)
acc_RC

In [ ]:
# Perceptron

P = Perceptron()
P.fit(X_train, y_train)
y_pred = P.predict(X_valid)

acc_P = round(accuracy_score(y_valid,y_pred) * 100, 2)
acc_P

In [ ]:
# Stochastic Gradient Descent
# https://scikit-learn.org/stable/modules/sgd.html#implementation-details

SGD = SGDClassifier(shuffle=True,average=True)
SGD.fit(X_train, y_train)
y_pred = SGD.predict(X_valid)

acc_SGD = round(accuracy_score(y_valid,y_pred) * 100, 2)
acc_SGD

#### KNN

In [ ]:
# KNN

knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_valid)

acc_knn = round(accuracy_score(y_valid,y_pred) * 100, 2)
acc_knn

#### SVC

In [ ]:
# Support Vector Classifier

SVC = SVC(probability=True)
SVC.fit(X_train, y_train)
y_pred = SVC.predict(X_valid)

acc_SVC = round(accuracy_score(y_valid,y_pred) * 100, 2)
acc_SVC

In [ ]:
# Linear SVC

linear_SVC = LinearSVC()
linear_SVC.fit(X_train,y_train)
linear_SVC.predict(X_valid)

acc_linear_SVC = round(accuracy_score(y_valid,y_pred) * 100, 2)
acc_linear_SVC

#### XGBoost

In [ ]:
# XGBoost

# xgb = XGBClassifier(n_estimators=1000, learning_rate=0.05, n_jobs=5)
# xgb.fit(X_train, y_train, 
#              early_stopping_rounds=5, 
#              eval_set=[(X_valid, y_valid)], 
#              verbose=False)

# y_pred = xgb.predict(X_valid)
# acc_xgb = round(accuracy_score(y_valid,y_pred) * 100, 2)
# acc_xgb

#### Discriminant Analysis

In [ ]:
# LinearDiscriminantAnalysis

LDA = LinearDiscriminantAnalysis()
LDA.fit(X_train,y_train)
LDA.predict(X_valid)

acc_LDA = round(accuracy_score(y_valid,y_pred) * 100, 2)
acc_LDA

In [ ]:
# QuadraticDiscriminantAnalysis

QDA = QuadraticDiscriminantAnalysis()
QDA.fit(X_train,y_train)
QDA.predict(X_valid)

acc_QDA = round(accuracy_score(y_valid,y_pred) * 100, 2)
acc_QDA

#### Naive Bayes

In [ ]:
# BernoulliNB

bernoulliNB = BernoulliNB()
bernoulliNB.fit(X_train,y_train)
bernoulliNB.predict(X_valid)

acc_bernoulliNB = round(accuracy_score(y_valid,y_pred) * 100, 2)
acc_bernoulliNB

In [ ]:
# GaussianNB

gaussianNB = GaussianNB()
gaussianNB.fit(X_train,y_train)
gaussianNB.predict(X_valid)

acc_gaussianNB = round(accuracy_score(y_valid,y_pred) * 100, 2)
acc_gaussianNB

### Compare model results

In [ ]:
models = pd.DataFrame({
    'Model': ['Decision Tree',"Extra Tree",'Random Forest','Support Vector', 'KNN', 'Logistic Regression', 
              'Stochastic Gradient Decent', 'Linear SVC', "Ada Boost Classifier", 
              "Bagging Classifier", "Passive Agressive Cl", "Ridge","Perceptron",
              'Gradient Boosting Classifier',"Extra Trees",
              "LinearDA","QuadraticDA","BernoulliNB","GaussianNB"],
    'Score': [acc_decision_tree,acc_extra_tree,acc_rfc, acc_SVC, acc_knn, acc_LG,
              acc_SGD, acc_linear_SVC, acc_AB, 
              acc_BC, acc_PAC, acc_RC, acc_P,
              acc_GB, acc_ETC,
             acc_LDA, acc_QDA, acc_bernoulliNB, acc_gaussianNB]})
sorted_by_score = models.sort_values(by='Score', ascending=False)

In [ ]:
#barplot using https://seaborn.pydata.org/generated/seaborn.barplot.html
sns.barplot(x='Score', y = 'Model', data = sorted_by_score, color = 'g')

#prettify using pyplot: https://matplotlib.org/api/pyplot_api.html
plt.title('Machine Learning Algorithm Accuracy Score \n')
plt.xlabel('Accuracy Score on validation data (%)')
plt.ylabel('Model')

In [ ]:
# cross-validation?

The top 3 models are: 
- Gradient Boosting Classifier
- Random Forest
- Bagging Classifier

Out of them, the Random Forest is the fastest one.
We are now going to find the best parameters for these 3 models using GridSearch.

## Hyperparameter tuning

We will be using the Grid Search for hyperparameter tuning here. We need to find the following best parameters for our Gradient Boosting model:
- learning rate
- max_depth
- min_samples_leaf
- max_featres
- n_estimators

In order to significantly reduce the massive computational cost of doing the brute force calculations, we will run the gridsearch on only two parameters at a time.
Reference: https://zlatankr.github.io/posts/2017/01/23/pump-it-up

In [ ]:
#!!! Takes too long!
# grid_search full
# GB = GradientBoostingClassifier(n_estimators=100, 
#                                 learning_rate=0.075,
#                                 max_depth=14,
#                                 max_features=1.0,
#                                 min_samples_leaf=16)


# param_grid = {"n_estimators" : [50,100, 150],
#               "learning_rate":[0.05, 0.025, 0.075, 0.01],
#              "max_depth" : [12,13,14], 
#               "min_samples_leaf":[14,15,16,17],
#              "max_features" : [0.5,0.3,0.7,1.0]}

# gs = GridSearchCV(estimator=GB,
#                   param_grid=param_grid,
#                   scoring='accuracy',
#                   cv=10,
#                   n_jobs=-1)

# gs.fit(X, y)

# print(gs.best_score_)
# print(gs.best_params_)
# print(gs.grid_scores_)

In [43]:
# Tuning for RF
sc = ss()
X = sc.fit_transform(X)

rfc = RandomForestClassifier(criterion='entropy', n_estimators = 50,random_state=42)

params = {"min_samples_split" : [4, 6, 8],
             "n_estimators" : [500, 700, 1000]}


grid_search = GridSearchCV(estimator=rfc, cv=4, param_grid=params, n_jobs=-1, verbose=5) # n_jobs=-1 = use all the CPU cores

grid_search.fit(X, y.values.ravel())

print(grid_search.best_score_)
print(grid_search.best_params_)

Fitting 4 folds for each of 9 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  8.3min
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed: 22.3min finished


0.7936531986531986
{'min_samples_split': 8, 'n_estimators': 1000}


In [15]:
# Tuning for LGB
sc = ss()
X = sc.fit_transform(X)

LGB = LGBMClassifier(objective='multiclass', num_threads=2, verbose=2, random_state=123)

params = {'num_iterations ': [100, 150, 200],
          'max_depth': [5, 8, 15],
          'learning_rate': [0.01, 0.75, 0.1, 0.2],
          'num_leaves' : [25, 40, 50]
         }

grid_search = GridSearchCV(estimator=LGB, cv=4, param_grid=params, n_jobs=-1, verbose=5) # n_jobs=-1 = use all the CPU cores

grid_search.fit(X, y.values.ravel())

print(grid_search.best_score_)
print(grid_search.best_params_)

Fitting 4 folds for each of 108 candidates, totalling 432 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   48.9s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  8.0min
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed: 13.1min
[Parallel(n_jobs=-1)]: Done 432 out of 432 | elapsed: 19.7min finished


0.7827609427609428
{'learning_rate': 0.75, 'max_depth': 15, 'num_iterations ': 100, 'num_leaves': 40}


In [ ]:
# randomized search full
GB = GradientBoostingClassifier(n_estimators=100, 
                                learning_rate=0.075,
                                max_depth=14,
                                max_features=1.0,
                                min_samples_leaf=16)


param_dist = {"n_estimators" : [50,100, 150],
              "learning_rate":[0.05, 0.025, 0.075, 0.01],
             "max_depth" : [12,13,14], 
              "min_samples_leaf":[14,15,16,17],
             "max_features" : [0.5,0.3,0.7,1.0]}

rs = RandomizedSearchCV(estimator=GB,
                  param_distributions=param_dist,
                  scoring='accuracy',
                  cv=10, n_iter=10, n_jobs=-1)

rs.fit(X, y)

print(rs.best_score_)
print(rs.best_params_)

RandomizedSearchCV result:

param_dist 
- "n_estimators" : [50,100, 150],
- "learning_rate":[0.05, 0.025, 0.075, 0.01],
- "max_depth" : [12,13,14], 
- "min_samples_leaf":[14,15,16,17],
- "max_features" : [0.5,0.3,0.7,1.0]

0.7967676767676768

{'n_estimators': 100, 'min_samples_leaf': 14, 'max_features': 0.5, 'max_depth': 14, 'learning_rate': 0.075}

In [ ]:
# randomized search min_samples_split
GB = GradientBoostingClassifier(n_estimators=100, 
                                learning_rate=0.075,
                                max_depth=14,
                                max_features=1.0,
                                min_samples_leaf=14)


param_dist = {"min_samples_split":[0.1,0.5,0.3,0.7,1.0]}

rs = RandomizedSearchCV(estimator=GB,
                  param_distributions=param_dist,
                  scoring='accuracy',
                  cv=10, n_iter=10, n_jobs=-1)

rs.fit(X, y)

print(rs.best_score_)
print(rs.best_params_)

## Variable importance

In [ ]:
X_df = pd.DataFrame(X)

In [ ]:
pd.concat((pd.DataFrame(X_df.columns, columns = ['variable']), 
           pd.DataFrame(GB.feature_importances_, columns = ['importance'])), 
          axis = 1).sort_values(by='importance', ascending = False)[:10]

## Retrain the chosen model on the whole train set

In [16]:
sc = ss()
X = sc.fit_transform(X)
X_test = sc.transform(X_test_final)

rfc = RandomForestClassifier(criterion='gini',min_samples_split=8, n_estimators=1000)

rfc.fit(X, y)     

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=8,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [43]:
sc = ss()
X = sc.fit_transform(X)

# GradientBoostingClassifier

GB = GradientBoostingClassifier(n_estimators=100, learning_rate=0.075, max_depth=14,max_features=1.0,min_samples_leaf=14,verbose=5)

GB.fit(X, y)     

      Iter       Train Loss   Remaining Time 
         1       49195.3374           18.62m
         2       46278.0771           19.49m
         3       43821.1036           19.67m
         4       41717.8833           19.72m
         5       39836.5003           19.57m
         6       38181.2752           19.36m
         7       36675.6249           19.26m
         8       35352.8606           19.13m
         9       34057.2438           19.03m
        10       32947.2188           18.81m
        11       31921.2694           18.66m
        12       31002.1300           18.50m
        13       30179.6605           18.23m
        14       29414.8409           17.95m
        15       28698.2535           17.75m
        16       28049.5921           17.47m
        17       27410.8015           17.25m
        18       26824.9707           16.98m
        19       26267.6517           16.72m
        20       25766.1268           16.45m
        21       25320.6267           16.17m
        2

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.075, loss='deviance', max_depth=14,
                           max_features=1.0, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=14, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=5,
                           warm_start=False)

In [79]:
# Hist GB
HGB = HistGradientBoostingClassifier(learning_rate=0.02, loss='categorical_crossentropy', 
                                               max_depth=8, min_samples_leaf=15)

HGB = HGB.fit(X_pca, y)

In [84]:
# Lightgbm
LGB = LGBMClassifier(objective='multiclass', learning_rate=0.75, num_iterations=100, 
                     num_leaves=40, random_state=123,max_depth=15)

LGB.fit(X_pca, y)

/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.75, max_depth=15,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_iterations=100, num_leaves=40,
               objective='multiclass', random_state=123, reg_alpha=0.0,
               reg_lambda=0.0, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

## Voting classifier

In [30]:
"""
You can combine your best predictors as a VotingClassifier, which can enhance the performance.

26/01/2020 - The hist_gradient / lgbm / gradient combination made the best result yet: 80.35%!
"""

estimators = [('HGB', HGB), ('LGB', LGB), ('GB', GB)]

ensemble = VotingClassifier(estimators, voting='soft')

ensemble.fit(X, y)

/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


      Iter       Train Loss   Remaining Time 
         1       49362.7959           18.43m
         2       46578.7639           19.08m
         3       44283.0241           19.08m
         4       42267.9223           18.93m
         5       40506.5750           18.85m
         6       38957.2751           18.67m
         7       37547.6901           18.56m
         8       36278.0766           18.42m
         9       35135.1664           18.26m
        10       34081.1956           18.10m
        11       33118.4529           18.00m
        12       32261.1496           17.81m
        13       31443.6473           17.62m
        14       30715.2200           17.40m
        15       30040.0994           17.18m
        16       29404.6845           16.97m
        17       28811.9159           16.75m
        18       28251.1060           16.56m
        19       27722.5996           16.37m
        20       27232.0737           16.16m
        21       26769.3164           15.95m
        2

VotingClassifier(estimators=[('HGB',
                              HistGradientBoostingClassifier(l2_regularization=0.0,
                                                             learning_rate=0.02,
                                                             loss='categorical_crossentropy',
                                                             max_bins=256,
                                                             max_depth=8,
                                                             max_iter=100,
                                                             max_leaf_nodes=31,
                                                             min_samples_leaf=15,
                                                             n_iter_no_change=None,
                                                             random_state=None,
                                                             scoring=None,
                                                             tol=1e-07,
       

## Submission

In [44]:
submission_df = pd.read_csv("SubmissionFormat.csv")

In [45]:
X_test = sc.transform(X_test_final)
submission_df['status_group']=GB.predict(X_test)

In [46]:
vals_to_replace = {2:'functional', 1:'functional needs repair', 0:'non functional'}

submission_df.status_group = submission_df.status_group.replace(vals_to_replace)

In [47]:
submission_df.to_csv("submission_TatianaSwrt_GB22.csv",sep=',', index=False)

## Possible future improvements
- Create a 'for' loop to automate the process of model selection
- Improve the code to deal with missing values to make it more efficient
- More feature ingeneering
- try to remove amount_tsh
- ? (Balazs) binary -> not unknown but false
- log transform to reduce skew: population, amount_tsh
- ? quality and rest - don’t make categories
- imbalanced -> oversampling with SMOTE -> didn’t help to Balazs
- xgboost -> feature importance (not sklearn)
- keras to balance classes
- try a different scaler

After the Feature Engineering Kaggle course I had these ideas:

- 